# Пучок микрозеркал TPC (визуализация 7 эллипсов)

В сборке фиксированы мини-зеркала (срезы стекловолокна диаметром 1 мм, эффективный диаметр среза ≈1.2 мм при угле ~34°). В конфиге задаются углы и высота центра каждого среза, после чего здесь строится 3D-сцена: поверхности эллипсов, их контуры, нормали и подписи с углами.


## Формат `bundle_config.txt`
- Глобальные параметры: `fiber_diameter` (мм), `pitch` (мм), `label_offset` (мм).
- Строки зеркал: `mirror=<имя>, cut=<угол_среза>, tilt=<азимут>, height=<z>[, x=<мм>, y=<мм>]`.
- `cut` — угол между нормалью плоскости среза и осью волокна (ось пучка — +Z). 0° — перпендикулярный срез (круг), при увеличении появляется эллипс с большим диаметром `fiber_diameter / cos(cut)`.
- `tilt` — азимут направления наклона нормали (0° вдоль +X, против часовой стрелки в плоскости XY).
- `height` — положение центра среза вдоль оси пучка.
- `x,y` — центр среза в плоскости XY; если не заданы, точки распределяются по окружности радиуса `pitch` (первый без координат станет центром, остальные равномерно по кругу).


In [17]:
import json
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional

import numpy as np
import plotly.graph_objects as go
from plotly.colors import qualitative


In [18]:
CONFIG_PATH = Path("bundle_config.txt")


@dataclass
class Mirror:
    name: str
    cut_deg: float
    tilt_deg: float
    height: float
    x: Optional[float] = None
    y: Optional[float] = None


@dataclass
class BundleConfig:
    fiber_diameter: float = 1.0
    pitch: float = 1.4
    label_offset: float = 0.6
    mirrors: List[Mirror] = None


def _parse_float(value: str, field: str, raw: str) -> float:
    try:
        return float(value)
    except ValueError as exc:
        raise ValueError(f"Не удалось прочитать {field} в строке: {raw}") from exc


def load_bundle_config(path: Path) -> BundleConfig:
    if not path.exists():
        raise FileNotFoundError(f"Файл конфигурации не найден: {path}")

    mirrors: List[Mirror] = []
    globals_cfg = {}

    for raw in path.read_text(encoding="utf-8").splitlines():
        clean = raw.split("#", 1)[0].strip()
        if not clean:
            continue

        if clean.startswith("mirror"):
            parts = [p.strip() for p in clean.split(",") if p.strip()]
            params = {}
            for part in parts:
                if "=" in part:
                    key, value = part.split("=", 1)
                    params[key.strip()] = value.strip()

            name = params.get("mirror") or params.get("name")
            if not name:
                raise ValueError(f"Строка без имени зеркала: {raw}")

            cut = _parse_float(params.get("cut", ""), "cut", raw)
            tilt = _parse_float(params.get("tilt", "0"), "tilt", raw)
            height = _parse_float(params.get("height", "0"), "height", raw)
            x = _parse_float(params["x"], "x", raw) if "x" in params else None
            y = _parse_float(params["y"], "y", raw) if "y" in params else None

            mirrors.append(Mirror(name=name, cut_deg=cut, tilt_deg=tilt, height=height, x=x, y=y))
        else:
            if "=" not in clean:
                continue
            key, value = clean.split("=", 1)
            globals_cfg[key.strip()] = float(value.strip())

    if not mirrors:
        raise ValueError("Не найдено ни одного зеркала в конфиге.")

    bundle = BundleConfig(
        fiber_diameter=float(globals_cfg.get("fiber_diameter", 1.0)),
        pitch=float(globals_cfg.get("pitch", globals_cfg.get("pitch_mm", 1.4))),
        label_offset=float(globals_cfg.get("label_offset", 0.6)),
        mirrors=mirrors,
    )
    return bundle


def assign_positions(cfg: BundleConfig) -> None:
    # Центр, если указан по имени и без координат
    for mirror in cfg.mirrors:
        if (mirror.x is None or mirror.y is None) and "center" in mirror.name.lower():
            mirror.x = 0.0
            mirror.y = 0.0

    missing = [m for m in cfg.mirrors if m.x is None or m.y is None]
    if missing:
        radius = cfg.pitch
        angles = np.linspace(0.0, 2 * np.pi, num=len(missing), endpoint=False)
        for mirror, ang in zip(missing, angles):
            mirror.x = radius * np.cos(ang)
            mirror.y = radius * np.sin(ang)


def ellipse_geometry(mirror: Mirror, cfg: BundleConfig, *, theta_samples: int = 80, radial_samples: int = 32):
    cut_rad = np.deg2rad(mirror.cut_deg)
    tilt_rad = np.deg2rad(mirror.tilt_deg)
    normal = np.array([np.sin(cut_rad) * np.cos(tilt_rad), np.sin(cut_rad) * np.sin(tilt_rad), np.cos(cut_rad)], dtype=float)
    normal = normal / np.linalg.norm(normal)
    center = np.array([mirror.x, mirror.y, mirror.height], dtype=float)

    axis = np.array([0.0, 0.0, 1.0])
    cos_theta = float(np.clip(np.dot(normal, axis), -1.0, 1.0))
    minor_r = cfg.fiber_diameter * 0.5
    major_r = minor_r / max(abs(cos_theta), 1e-6)

    tangent = axis - cos_theta * normal
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([1.0, 0.0, 0.0]))
    if np.linalg.norm(tangent) < 1e-8:
        tangent = np.cross(normal, np.array([0.0, 1.0, 0.0]))
    e1 = tangent / np.linalg.norm(tangent)
    e2 = np.cross(normal, e1)

    theta = np.linspace(0.0, 2 * np.pi, theta_samples)
    radii = np.linspace(0.0, 1.0, radial_samples)
    T, R = np.meshgrid(theta, radii, indexing="ij")

    pts = center + (major_r * R * np.cos(T))[..., None] * e1 + (minor_r * R * np.sin(T))[..., None] * e2
    boundary = center + major_r * np.cos(theta)[:, None] * e1 + minor_r * np.sin(theta)[:, None] * e2
    label = center + normal * cfg.label_offset

    flat_points = np.column_stack((pts[..., 0].ravel(), pts[..., 1].ravel(), pts[..., 2].ravel()))

    return {
        "center": center,
        "normal": normal,
        "major_r": major_r,
        "minor_r": minor_r,
        "X": pts[..., 0],
        "Y": pts[..., 1],
        "Z": pts[..., 2],
        "boundary": boundary,
        "label": label,
        "flat_points": flat_points,
    }


def build_geometry(cfg: BundleConfig):
    geometries = []
    for mirror in cfg.mirrors:
        geom = ellipse_geometry(mirror, cfg)
        geometries.append((mirror, geom))
    return geometries


def describe(cfg: BundleConfig, geometries):
    print("Параметры зеркал (мм):")
    for mirror, geom in geometries:
        major = geom["major_r"] * 2
        minor = geom["minor_r"] * 2
        print(
            f"- {mirror.name}: cut={mirror.cut_deg:.1f}°, tilt={mirror.tilt_deg:.1f}°, z={mirror.height:.2f}; "
            f"pos=({mirror.x:.2f}, {mirror.y:.2f}); axes={major:.2f} × {minor:.2f}"
        )


print("Функции загружены. Выполните следующую ячейку для построения.")


Функции загружены. Выполните следующую ячейку для построения.


In [19]:
cfg = load_bundle_config(CONFIG_PATH)
assign_positions(cfg)
geometries = build_geometry(cfg)
describe(cfg, geometries)

bbox = np.vstack([g["flat_points"] for _, g in geometries])
bbox_min = bbox.min(axis=0)
bbox_max = bbox.max(axis=0)
padding = cfg.fiber_diameter
print(
    f"Диапазон сцены: X[{bbox_min[0]:.2f}, {bbox_max[0]:.2f}], "
    f"Y[{bbox_min[1]:.2f}, {bbox_max[1]:.2f}], Z[{bbox_min[2]:.2f}, {bbox_max[2]:.2f}] (мм)"
)


Параметры зеркал (мм):
- ring1: cut=45.0°, tilt=281.0°, z=0.00; pos=(1.00, 0.00); axes=1.41 × 1.00
- ring2: cut=45.0°, tilt=286.5°, z=0.85; pos=(0.50, 0.87); axes=1.41 × 1.00
- ring3: cut=45.0°, tilt=253.5°, z=0.85; pos=(-0.50, 0.87); axes=1.41 × 1.00
- ring4: cut=45.0°, tilt=259.0°, z=0.00; pos=(-1.00, 0.00); axes=1.41 × 1.00
- ring5: cut=45.0°, tilt=264.5°, z=-0.85; pos=(-0.50, -0.87); axes=1.41 × 1.00
- ring6: cut=45.0°, tilt=275.5°, z=-0.85; pos=(0.50, -0.87); axes=1.41 × 1.00
- center: cut=45.0°, tilt=270.0°, z=0.00; pos=(0.00, 0.00); axes=1.41 × 1.00
Диапазон сцены: X[-1.50, 1.50], Y[-1.37, 1.37], Z[-1.35, 1.35] (мм)


In [21]:
colors = qualitative.Dark24 + qualitative.Plotly + qualitative.Alphabet
fig = go.Figure()

incoming_dir = np.array([0.0, 0.0, -1.0])  # ???????? ??? ?????? ????
ray_len_in = max(cfg.label_offset * 1.5, cfg.fiber_diameter * 8.0)
ray_len_out = ray_len_in
extra_points = []

for idx, (mirror, geom) in enumerate(geometries):
    color = colors[idx % len(colors)]
    fig.add_trace(
        go.Surface(
            x=geom["X"],
            y=geom["Y"],
            z=geom["Z"],
            surfacecolor=np.zeros_like(geom["X"]),
            cmin=0,
            cmax=1,
            colorscale=[[0, color], [1, color]],
            showscale=False,
            opacity=0.7,
            name=mirror.name,
            customdata=np.stack(
                [
                    np.full_like(geom["X"], mirror.cut_deg),
                    np.full_like(geom["X"], mirror.tilt_deg),
                    np.full_like(geom["X"], mirror.height),
                ],
                axis=-1,
            ),
            hovertemplate=(
                mirror.name
                + "<br>cut=%{customdata[0]:.1f} deg"
                + "<br>tilt=%{customdata[1]:.1f} deg"
                + "<br>z=%{customdata[2]:.2f} mm<extra></extra>"
            ),
        )
    )

    boundary = geom["boundary"]
    fig.add_trace(
        go.Scatter3d(
            x=boundary[:, 0],
            y=boundary[:, 1],
            z=boundary[:, 2],
            mode="lines",
            line=dict(color=color, width=6),
            name=f"{mirror.name} contour",
            showlegend=False,
        )
    )

    c = geom["center"]
    label = geom["label"]
    fig.add_trace(
        go.Scatter3d(
            x=[c[0], label[0]],
            y=[c[1], label[1]],
            z=[c[2], label[2]],
            mode="lines",
            line=dict(color=color, width=3, dash="dash"),
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=[label[0]],
            y=[label[1]],
            z=[label[2]],
            mode="text",
            text=[
                f"{mirror.name}\ncut={mirror.cut_deg:.1f} deg\ntil={mirror.tilt_deg:.1f} deg\nz={mirror.height:.2f} mm"
            ],
            textposition="top center",
            textfont=dict(color=color, size=12),
            showlegend=False,
        )
    )

    # ????????? ???? ????? ?????????? ?????????: R = I - 2 nn^T
    n = geom["normal"] / np.linalg.norm(geom["normal"])
    reflect_mat = np.eye(3) - 2.0 * np.outer(n, n)
    outgoing_dir = reflect_mat @ incoming_dir
    outgoing_dir = outgoing_dir / np.linalg.norm(outgoing_dir)

    incoming_start = c - incoming_dir * ray_len_in  # ????? ???? ???????
    incoming_pts = np.vstack([incoming_start, c])
    outgoing_end = c + outgoing_dir * ray_len_out
    outgoing_pts = np.vstack([c, outgoing_end])
    extra_points.extend([incoming_start, outgoing_end])

    fig.add_trace(
        go.Scatter3d(
            x=incoming_pts[:, 0],
            y=incoming_pts[:, 1],
            z=incoming_pts[:, 2],
            mode="lines",
            line=dict(color="#d62728", width=5),
            name=f"{mirror.name} incoming",
            showlegend=(idx == 0),
        )
    )
    fig.add_trace(
        go.Scatter3d(
            x=outgoing_pts[:, 0],
            y=outgoing_pts[:, 1],
            z=outgoing_pts[:, 2],
            mode="lines",
            line=dict(color="#2ca02c", width=5),
            name=f"{mirror.name} reflected",
            showlegend=(idx == 0),
        )
    )

# ????????? ???????? ????? ? ?????? ?????
bbox_min_ext = bbox_min.copy()
bbox_max_ext = bbox_max.copy()
if extra_points:
    p = np.array(extra_points)
    bbox_min_ext = np.minimum(bbox_min_ext, p.min(axis=0))
    bbox_max_ext = np.maximum(bbox_max_ext, p.max(axis=0))

fig.update_layout(
    title="????? ???????????: ????? ? ????",
    scene=dict(
        xaxis=dict(title="X (mm)", range=[bbox_min_ext[0] - padding, bbox_max_ext[0] + padding]),
        yaxis=dict(title="Y (mm)", range=[bbox_min_ext[1] - padding, bbox_max_ext[1] + padding]),
        zaxis=dict(title="Z (mm)", range=[bbox_min_ext[2] - padding, bbox_max_ext[2] + padding]),
        aspectmode="cube",
    ),
    margin=dict(l=0, r=0, t=50, b=0),
)
fig.show()

